<a href="https://colab.research.google.com/github/jaehyun0220/Colab/blob/master/jigsaw_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 데이터 활용에 필요한 기본 패키지 로딩
import sys #access to system parameters 
print("Python version: {}". format(sys.version))

import pandas as pd
print("pandas version: {}". format(pd.__version__))

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import tensorflow as tf
print("tensorflow version: {}".format(tf.__version__))

import keras
print("keras version: {}".format(keras.__version__))

import os
#import io

Python version: 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
pandas version: 0.24.2
scikit-learn version: 0.21.1
NumPy version: 1.16.3
tensorflow version: 1.13.1
keras version: 2.2.4


In [0]:
#데이터 전처리 관련 라이브러리 로드
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ShuffleSplit

# Deep Learning Model 로드
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation 
from keras.wrappers.scikit_learn import KerasClassifier

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#HyperParameter Tuning을 위한 라이브러리 로드
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#모델 평가를 위한 라이브러리 로드
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn import model_selection

#수학 & 통계 관련 라이브러리 로드
import scipy.stats as st
from collections import Counter

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Auth 인증 및 Google Drive 활용 Data load
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# Google Drive 내 Custom Class 경로 지정
import sys
sys.path.insert(0, '/gdrive/My Drive/CustomClasses')

Mounted at /gdrive


In [0]:
# from google.colab import files
# files.upload()
# !cp ./kaggle.json /root/.kaggle/kaggle.json
# !cp ./kaggle.json /gdrive/My\ Drive/kaggle/kaggle.json

In [34]:
# kaggle 실행을 위한 json file copy & 권한 부여
!cp /gdrive/My\ Drive/kaggle/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json
# !cat /gdrive/My\ Drive/CustomClasses/.kaggle/kaggle.json
# !kaggle competitions list

{"username":"claypark","key":"bb187008d01e8abfeaabdb973ebfa7a9"}

In [0]:
# Kaggle Competition Data Download - JIGSAW competition

# Directory 생성 후 데이터 다운로드
# !mkdir /gdrive/My\ Drive/kaggle/jigsaw  
# !kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -p /gdrive/My\ Drive/kaggle/jigsaw 

# 파일 확인 후 각각 unzip 후 zip 파일 삭제
# !ls /gdrive/My\ Drive/kaggle/jigsaw
# !unzip /gdrive/My\ Drive/kaggle/jigsaw/train.csv.zip -d /gdrive/My\ Drive/kaggle/jigsaw
# !unzip /gdrive/My\ Drive/kaggle/jigsaw/test.csv.zip -d /gdrive/My\ Drive/kaggle/jigsaw
# !unzip /gdrive/My\ Drive/kaggle/jigsaw/sample_submission.csv.zip -d /gdrive/My\ Drive/kaggle/jigsaw
# !rm /gdrive/My\ Drive/kaggle/jigsaw/*.zip

In [52]:
!ls /gdrive/My\ Drive/kaggle/jigsaw

sample_submission.csv  test.csv  train.csv
